In [3]:
import os
import sys
import random
import librosa
import numpy as np
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from sklearn.metrics import silhouette_score

import torch

# ===== настройки =====

random.seed(42)

paths = {
    "drones": "../data/drones",
    "not_drones": "../data/not_drones",
}

In [4]:
import sys, os

BEATS_DIR = "./beats"   # путь к скачанной папке
BEATS_DIR = os.path.abspath(BEATS_DIR)
sys.path.append(BEATS_DIR)

from BEATs import BEATs, BEATsConfig

# путь до чекпойнта модели (.pt), скачанного из релизов BEATs
# пример: "BEATs_iter3_plus_AS2M.pt" или "BEATs_iter3.pt"
MODEL_PATH = os.path.join(BEATS_DIR, "BEATs_iter3.pt")  # <<< ПОМЕНЯЙ НА СВОЙ ФАЙЛ

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# ===== файлы =====

def collect_files(path):
    fs = []
    for root, _, files in os.walk(path):
        for f in files:
            if f.lower().endswith((".wav", ".mp3")):
                fs.append(os.path.join(root, f))
    return sorted(fs)

drones_all = collect_files(paths["drones"])
not_drones_all = collect_files(paths["not_drones"])

# берем часть (1/50)
drones_sel = drones_all[: len(drones_all) // 50]
not_drones_sel = not_drones_all[: len(not_drones_all) // 50]

files = drones_sel + not_drones_sel
labels = [1] * len(drones_sel) + [0] * len(not_drones_sel)

print("дронов:", len(drones_sel))
print("не дронов:", len(not_drones_sel))
print("всего:", len(files))


device: cpu
дронов: 1411
не дронов: 1304
всего: 2715


In [6]:
# ===== модель beats =====

print("модель...")

checkpoint = torch.load(MODEL_PATH, map_location=device)
cfg = BEATsConfig(checkpoint["cfg"])
beats_model = BEATs(cfg)
beats_model.load_state_dict(checkpoint["model"])
beats_model.to(device)
beats_model.eval()

print("ok")

# ===== загрузка аудио =====

def load_audio(path):
    if not os.path.exists(path):
        return None
    try:
        audio, sr = librosa.load(path, sr=None, mono=True)
        return audio, sr
    except Exception:
        return None

print("загрузка...")
loaded = []
with ThreadPoolExecutor(max_workers=8) as ex:
    for item in tqdm(ex.map(load_audio, files), total=len(files), desc="аудио"):
        if item is not None:
            loaded.append(item)

print("загружено:", len(loaded))

модель...


c:\Users\Samat\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\utils\weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


ok
загрузка...


аудио: 100%|██████████| 2715/2715 [00:05<00:00, 487.83it/s] 

загружено: 2715


In [7]:

# ===== эмбеддинги beats =====

print("эмбеддинги...")
embeddings = []
batch_size = 4
processed = 0

for i in range(0, len(loaded), batch_size):
    batch = loaded[i:i + batch_size]
    if not batch:
        continue

    # ресемпл до 16к
    waves = []
    for audio, sr in batch:
        if sr != 16000:
            audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)
        waves.append(audio)

    # выравнивание длины
    max_len = max(len(w) for w in waves)
    waves_pad = [librosa.util.fix_length(w, size=max_len) for w in waves]

    audio_tensor = torch.tensor(waves_pad, dtype=torch.float32, device=device)  # (b, t)
    padding_mask = torch.zeros(audio_tensor.shape, dtype=torch.bool, device=device)

    with torch.no_grad():
        rep = beats_model.extract_features(audio_tensor, padding_mask=padding_mask)[0]
        # rep: (b, t, dim) или (b, dim) в зав-ти от версии
        if rep.dim() == 3:
            rep_pooled = rep.mean(dim=1)  # (b, dim)
        else:
            rep_pooled = rep  # (b, dim)

    for vec in rep_pooled.cpu().numpy():
        embeddings.append(vec)
        processed += 1
        if processed % 100 == 0:
            print("обработано:", processed)

print("эмбеддингов:", len(embeddings))

эмбеддинги...


C:\Users\Samat\AppData\Local\Temp\ipykernel_9328\1085348468.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:256.)
  audio_tensor = torch.tensor(waves_pad, dtype=torch.float32, device=device)  # (b, t)


обработано: 100
обработано: 200
обработано: 300
обработано: 400
обработано: 500
обработано: 600
обработано: 700
обработано: 800
обработано: 900
обработано: 1000
обработано: 1100
обработано: 1200
обработано: 1300
обработано: 1400
обработано: 1500
обработано: 1600
обработано: 1700
обработано: 1800
обработано: 1900
обработано: 2000
обработано: 2100
обработано: 2200
обработано: 2300
обработано: 2400
обработано: 2500
обработано: 2600
обработано: 2700
эмбеддингов: 2715


In [8]:
if len(embeddings) == 0:
    print("нет эмбеддингов — ошибка")
    raise SystemExit()

# ===== массивы =====

X = np.vstack(embeddings)
y = np.array(labels[: len(X)])

print("X форма:", X.shape)
print("y форма:", y.shape)

# ===== метрика =====

score = silhouette_score(X, y)
print("sil:", score)

# ===== сохранение =====

os.makedirs("../embeddings", exist_ok=True)
save_path = "../embeddings/beats.npz"
np.savez(save_path, X=X, y=y, score=score)

print("сохранено:", save_path)

X форма: (2715, 768)
y форма: (2715,)
sil: 0.46975770592689514
сохранено: ../embeddings/beats.npz
